# import each week of data in

goff.csv
mariota.csv
kenny_p.csv
aaron_rogers.csv
r_wilson.csv
matty_ice.csv

In [1]:
import pandas as pd

#df = pd.read_csv("qb_data/mahomes.csv")
#df = pd.read_csv("qb_data/jalen_hurts.csv")
#df = pd.read_csv("qb_data/josh_allen.csv")
#df = pd.read_csv("qb_data/joe_burrow.csv")
#df = pd.read_csv("qb_data/justin_herbert.csv")
#df = pd.read_csv("qb_data/lamar_jackson.csv")
#df = pd.read_csv("qb_data/tua.csv")
#df = pd.read_csv("qb_data/trevor_lawrence.csv")
#df = pd.read_csv("qb_data/geno_smith.csv")
#df = pd.read_csv("qb_data/carr.csv")
#df = pd.read_csv("qb_data/tom_brady.csv")
#df = pd.read_csv("qb_data/jimmy_g.csv")
#df = pd.read_csv("qb_data/kirk_cousins.csv")
#df = pd.read_csv("qb_data/goff.csv")
#df = pd.read_csv("qb_data/mariota.csv")
#df = pd.read_csv("qb_data/kenny_p.csv")
#df = pd.read_csv("qb_data/daniel_jones.csv")
#df = pd.read_csv("qb_data/aaron_rogers.csv")
#df = pd.read_csv("qb_data/tannehill.csv")
#df = pd.read_csv("qb_data/mac_jones.csv")
#df = pd.read_csv("qb_data/r_wilson.csv")
df = pd.read_csv("qb_data/matty_ice.csv")
#df = pd.read_csv("qb_data/dalton.csv")

###### new approach -- look at data week by week
train set is previous weeks
test set is current week

Based on EDA, we care about 6 QB stats:
1. Completions
2. Passing Yards
3. Passing TDs
4. Rushing Yards
5. Rushing TDs
6. Points

We will run linear regression on these stats with the target being points scored.

In [2]:
# examine data
df.describe()
df

,WEEK,OPP,CMP,ATT,PCT,YDS,AVG,TD,INT,RAT,ATT.1,YDS.1,AVG.1,TD.1,FPTS/G,FPTS
0,1,HOU,32,50,64.0,352,7.0,1,1,83.08,4,12,3.0,0,15.3,15.28
1,2,JAX,16,30,53.3,195,6.5,0,3,34.03,0,0,0.0,0,1.8,1.80
2,3,KC,27,37,73.0,222,6.0,2,0,105.91,3,4,1.3,0,15.3,15.28
3,4,TEN,27,37,73.0,356,9.6,2,1,109.74,2,-4,-2.0,0,17.8,17.84
4,5,DEN,26,41,63.4,251,6.1,0,2,60.11,1,1,1.0,0,6.1,6.14
5,6,JAX,42,58,72.4,389,6.7,3,0,107.61,1,-4,-4.0,0,27.2,27.16
6,7,TEN,33,44,75.0,243,5.5,1,2,76.23,1,2,2.0,0,9.9,9.92
7,10,LV,21,28,75.0,222,7.9,1,0,109.52,4,38,9.5,1,22.7,22.68
8,11,PHI,23,32,71.9,213,6.7,0,0,89.71,1,1,1.0,0,8.6,8.62
9,12,PIT,22,34,64.7,199,5.9,1,1,77.94,4,22,5.5,0,10.2,10.16


In [3]:
# check to make sure QB meets min 8 games played criteria
len(df.WEEK)

10

In [4]:
df.mean()
df

/tmp/ipykernel_20616/4005303691.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.mean()


,WEEK,OPP,CMP,ATT,PCT,YDS,AVG,TD,INT,RAT,ATT.1,YDS.1,AVG.1,TD.1,FPTS/G,FPTS
0,1,HOU,32,50,64.0,352,7.0,1,1,83.08,4,12,3.0,0,15.3,15.28
1,2,JAX,16,30,53.3,195,6.5,0,3,34.03,0,0,0.0,0,1.8,1.80
2,3,KC,27,37,73.0,222,6.0,2,0,105.91,3,4,1.3,0,15.3,15.28
3,4,TEN,27,37,73.0,356,9.6,2,1,109.74,2,-4,-2.0,0,17.8,17.84
4,5,DEN,26,41,63.4,251,6.1,0,2,60.11,1,1,1.0,0,6.1,6.14
5,6,JAX,42,58,72.4,389,6.7,3,0,107.61,1,-4,-4.0,0,27.2,27.16
6,7,TEN,33,44,75.0,243,5.5,1,2,76.23,1,2,2.0,0,9.9,9.92
7,10,LV,21,28,75.0,222,7.9,1,0,109.52,4,38,9.5,1,22.7,22.68
8,11,PHI,23,32,71.9,213,6.7,0,0,89.71,1,1,1.0,0,8.6,8.62
9,12,PIT,22,34,64.7,199,5.9,1,1,77.94,4,22,5.5,0,10.2,10.16


In [5]:
# create x df with 6 relevant rows
x_stats = df.drop(['WEEK', 'OPP', 'ATT', 'PCT', 'AVG', 'INT', 'RAT', 'ATT.1', 'AVG.1', 'FPTS/G'], axis=1)

# isolate y target
y_points = df.iloc[:,15]

# create train split
train_x = x_stats[:5]
train_y = y_points[:5]

# create test split
test_x = x_stats[6:-1]
test_y = y_points[6:-1]

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# run linear regression on training data
reg = LinearRegression()
reg.fit(train_x, train_y)

# check accuracy with test data
mean_squared_error(test_y, reg.predict(test_x))

0.11717879473023853

In [6]:
# lasso

from sklearn.linear_model import Lasso

reg = Lasso(alpha=5)    #alpha is size of threshold, total weight of abs val of our features can't exceed alpha
reg.fit(train_x, train_y)
mean_squared_error(test_y, reg.predict(test_x))

5.042974985309743

In [7]:
# Ridge
from sklearn.linear_model import Ridge

reg = Ridge(alpha=5)    #alpha is size of threshold, total weight of abs val of our features can't exceed alpha
reg.fit(train_x, train_y)
mean_squared_error(test_y, reg.predict(test_x))

2.3314677008153444

In [8]:
# divide data set into training, validation, and test
# Grid Search -- create grid of alpha and beta hyperparams
# Cross validation -- move validation portion around while training on data set

from sklearn.model_selection import GridSearchCV  #GriSearchCV becomes regressor instead of ridge
params = {'alpha': [0.001, 0.01, 0.1, 1, 2, 5, 10, 100, 1000]}
ridge = Ridge()
reg = GridSearchCV(ridge, params, scoring='neg_mean_squared_error',
                 cv=5)

reg.fit(train_x, train_y)
reg.best_params_
reg.best_score_
mean_squared_error(test_y, reg.predict(test_x))
# predict
reg.predict(test_x)

array([10.86766299, 18.84570219,  8.37225534])

In [9]:
# check test data
test_x

,CMP,YDS,TD,YDS.1,TD.1,FPTS
6,33,243,1,2,0,9.92
7,21,222,1,38,1,22.68
8,23,213,0,1,0,8.62


In [10]:
# double the values in their most recent game to weight the data towards the trend of their play
#replacement = test_x.iloc[-1,:].mul(2)
#test_x.iloc[-1,:] = replacement
# sum values
#future = test_x.sum(axis=0)
# divide by size + 1 to account for doubling values of recent game
#future = future.div(len(test_x.CMP) + 1)

In [11]:
# version with no doubled value of recent game
future = test_x.sum(axis=0)
future = future.div(len(test_x.CMP))

In [12]:
predict_upcoming_week = test_x.append(future, ignore_index=True)
predict_upcoming_week

/tmp/ipykernel_20616/3164448740.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predict_upcoming_week = test_x.append(future, ignore_index=True)


,CMP,YDS,TD,YDS.1,TD.1,FPTS
0,33.000000,243.0,1.000000,2.000000,0.000000,9.92
1,21.000000,222.0,1.000000,38.000000,1.000000,22.68
2,23.000000,213.0,0.000000,1.000000,0.000000,8.62
3,25.666667,226.0,0.666667,13.666667,0.333333,13.74


In [13]:
# predict with new avg column and take new value as prediction for upcoming week performance

expected_performance = reg.predict(predict_upcoming_week)

expected_performance[-1]
#expected_performance


12.695206839944905